Imports

In [33]:
import requests
import pymongo
import time
import pandas as pd
from secret import POLYGON_API_KEY
from global_variables import *

client = mongo_connect(ALGO_TRADING_DB_STRING)
database = get_database(client, 'algo_trading')
basic_data_collection = get_collection(database, 'basic_data')


Gets market summary from polygon api given dates
input: 
    Dates: a list of dates in isoformat (required)
outputs:
    data_list: a list containg dictionaries with the following structure:
        Date: date the data was pulled from
        Data: a dictionary filled with a seperate dictionary for each ticker with the following structure:
            {stock symbol}:
                v: volume (float)
                vw: volume weighted price (float)
                o: open price (float)
                c: close price (float)
                l: low price (float)
                h: high price (float)
                n: number of transactions (int)
                t: unix milliseconds of close time (long)

In [26]:
def get_stock_data(dates):
    data_list = []
    for i in range(len(dates)):
        date = dates[i]
        daily_data = requests.get(f'https://api.polygon.io/v2/aggs/grouped/locale/us/market/stocks/{date}?adjusted=true&apiKey={POLYGON_API_KEY}')
        daily_data = daily_data.json()
        if daily_data['status'] == 'ERROR': 
            time.sleep(65)
            i -= 1
            continue
        if daily_data['queryCount'] == 0: continue
        daily_data_results = daily_data['results']
        daily_data_dictionary = to_stock_dictionary(daily_data_results)
        data_list.append({'Date': date, 'Data': daily_data_dictionary})
        print(f'percent done: {(i / len(dates)) * 100}')
    return data_list

Adds a key to a dictionary and returns the dictionary
inputs:
    key: the key to add to the dictionary (required)
    value: the value to add to the given key (required)
    dic: the dictionary to add the key/value pair to (optional, will add the key to an empty dictionary by default)
output:
    dic: the given dictionary with the key/value pair added

In [3]:
def add_key(key, value, dic = {}):
    dic[key] = value
    return dic

Replaces documents in a mongodb collection that have the same date
inputs:
    collection: the collection to replace documents in (required)
    new_docs: an array containing the documents to replace with (required)
    many: a boolean (optional, default false), false means each document in new_docs will replace at most one document in collection with the same date. True means all documents in collection sharing a date with a document in new_docs will be replace (many -> one)
output: none

In [25]:
def replace_by_date(collection, new_docs, many = False):
    if many == False:
        for doc in new_docs:
            query_filter = {'Date': doc['Date']}
            collection.replace_one(query_filter, doc)
    else:
        for doc in new_docs:
            query_filter = {'Date': doc['Date']}
            collection.delete_many(query_filter)
            collection.insert_one(doc)

Add the percent change to a document containg daily stock data
inputs:
    collection: the collection to change documents of (required)
    documents: a cursor containg the documents to change (required)
output:
    none, modifies collection in-place

In [24]:
def add_percent_change(collection, documents):
    for doc in documents:
        date = doc['Date']
        data = doc['Data']
        for ticker in data.keys():
            prev_price = data[ticker]['o']
            curr_price = data[ticker]['c']
            price_change_percent = ((curr_price / prev_price) - 1) * 100
            data[ticker]['percent change'] = price_change_percent
        update_query = {'Date': date}
        update_operator = {'$set':
                {'Data': data}
        }
        collection.update_one(update_query, update_operator)

In [23]:
def remove_suffixes(stock_array, suffixes = []):
    i = 0
    while i < len(stock_array):
        ticker = stock_array[i]['T']
        if '.' not in ticker:
            i += 1
            continue
        suffix = ticker.split('.')[1]
        if suffix in suffixes:
            del stock_array[i]
        else:
            i += 1
    return stock_array

filters out documents with the given suffixes 
inputs:
    collection: the mongodb collection to replace documents in (by document date)
    docs: a cursor containing the documents to be filtered
    suffixes: a list of suffixes to be filtered

In [27]:
def filter_suffixes(collection, docs, suffixes):
    new_docs = []
    for doc in docs:
        stock_array = to_stock_array(doc['Data'])
        stock_array = remove_suffixes(stock_array, suffixes)
        stock_dictionary = to_stock_dictionary(stock_array)
        doc['Data'] = stock_dictionary
        new_docs.append(doc)
    replace_by_date(collection, new_docs)


In [35]:
def remove_warrants(stock_array):
    i = 0
    while i < len(stock_array):
        symbol = stock_array[i]['T']
        if len(symbol) > 4 and symbol[4] == 'W' and '.' not in symbol and 'p' not in symbol:
            del stock_array[i]
        else:
            i += 1
    return stock_array
                

In [29]:
def filter_warrants(collection, docs):
    new_docs = []
    for doc in docs:
        stock_array = to_stock_array(doc['Data'])
        stock_array = remove_warrants(stock_array)
        stock_dictionary = to_stock_dictionary(stock_array)
        doc['Data'] = stock_dictionary
        new_docs.append(doc)
    return new_docs

In [ ]:
no_warrants = filter_warrants(basic_data_collection, basic_data_collection.find({'Date': '2025-04-09'}))
a = 1